# Student Loan Risk with Deep Learning

In [18]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path
from tensorflow.keras.models import load_model

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [3]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [4]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [5]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [6]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"].values

# Display a sample of y
print(y[:5]) 

[0 0 0 1 0]


In [7]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop(columns=["credit_ranking"])

# Review the features DataFrame
print(X.head())

   payment_history  location_parameter  stem_degree_score  gpa_ranking  \
0              7.4                0.70               0.00          1.9   
1              7.8                0.88               0.00          2.6   
2              7.8                0.76               0.04          2.3   
3             11.2                0.28               0.56          1.9   
4              7.4                0.70               0.00          1.9   

   alumni_success  study_major_code  time_to_completion  \
0           0.076              11.0                34.0   
1           0.098              25.0                67.0   
2           0.092              15.0                54.0   
3           0.075              17.0                60.0   
4           0.076              11.0                34.0   

   finance_workshop_score  cohort_ranking  total_loan_score  \
0                  0.9978            3.51              0.56   
1                  0.9968            3.20              0.68   
2          

### Step 3: Split the features and target sets into training and testing datasets.


In [8]:
# Split the preprocessed data into a training and testing dataset
# Split the data into training and testing sets (80% train, 20% test)
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

print(f"X_train shape: {X_train.shape}, X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}, y_test shape: {y_test.shape}")

X_train shape: (1279, 11), X_test shape: (320, 11)
y_train shape: (1279,), y_test shape: (320,)


### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [9]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
scaler.fit(X_train)

# Transform the training data
X_train_scaled = scaler.transform(X_train)

# Transform the testing data using the same scaler
X_test_scaled = scaler.transform(X_test)

# Display a sample of the scaled training data
print(X_train_scaled[:5]) 


[[-0.73307913  0.6648928  -1.25704443 -0.3204585  -0.45362151 -0.74240736
  -0.6455073   0.24000129  0.98846046  0.0630946  -0.87223395]
 [ 1.06774091 -0.62346154  1.52314768  0.60886277 -0.36954631 -1.12518952
  -1.11200285  0.18789883 -1.7535127  -0.17390392 -0.77978452]
 [-1.74604041 -1.07158479 -1.35814232 -0.53491726 -0.78992229  1.07580793
   0.53628144 -2.67773653  2.32756363  0.77409018  3.28799021]
 [-0.62052788  0.49684658 -1.05484864 -0.0345135  -0.20139592  0.11885252
   1.18937522  0.37546769  1.24352773 -0.76640023 -0.6873351 ]
 [-0.50797663  0.60887739 -1.00429969 -0.53491726  0.26101766 -0.74240736
  -0.7077067  -0.33312578 -0.09557544 -0.47015208 -0.77978452]]


---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [10]:
# Define the the number of inputs (features) to the model
input_dim = X_train_scaled.shape[1]

# Review the number of features
print(f"Number of input features: {input_dim}")

Number of input features: 11


In [11]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = 16
print(f"Number of hidden nodes in the first hidden layer: {hidden_nodes_layer1}")
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = 8
print(f"Number of hidden nodes in the second hidden layer: {hidden_nodes_layer2}")
# Define the number of neurons in the output layer
output_neurons = 1 
print(f"Number of neurons in the output layer: {output_neurons}")

Number of hidden nodes in the first hidden layer: 16
Number of hidden nodes in the second hidden layer: 8
Number of neurons in the output layer: 1


In [12]:
# Create the Sequential model instance
model = Sequential([
    Dense(hidden_nodes_layer1, activation='relu', input_dim=input_dim),
    Dense(hidden_nodes_layer2, activation='relu'),
    Dense(output_neurons, activation='sigmoid')  # Sigmoid for binary classification
])

# Add the first hidden layer
model.add(Dense(hidden_nodes_layer1, activation='relu', input_dim=input_dim))

# Add the second hidden layer
model.add(Dense(hidden_nodes_layer2, activation='relu'))

# Add the output layer to the model specifying the number of output neurons and activation function
model.add(Dense(output_neurons, activation='sigmoid')) 

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
# Display the Sequential model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 16)             │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             9 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 16)             │            32 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 514 (2.01 KB)

 Trainable params: 514 (2.01 KB)

 Non-trainable params: 0 (0.00 B)

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [14]:
# Compile the Sequential model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Display a message to confirm successful compilation
print("Model compiled successfully!")

Model compiled successfully!


In [15]:
# Fit the model using 50 epochs and the training data
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=10, validation_data=(X_test_scaled, y_test), verbose=1)

Epoch 1/50
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5118 - loss: 0.6951 - val_accuracy: 0.5188 - val_loss: 0.6927
Epoch 2/50
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 571us/step - accuracy: 0.5505 - loss: 0.6891 - val_accuracy: 0.5188 - val_loss: 0.6916
Epoch 3/50
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 567us/step - accuracy: 0.5332 - loss: 0.6875 - val_accuracy: 0.5188 - val_loss: 0.6673
Epoch 4/50
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 551us/step - accuracy: 0.6307 - loss: 0.6456 - val_accuracy: 0.7312 - val_loss: 0.5880
Epoch 5/50
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 552us/step - accuracy: 0.7284 - loss: 0.5762 - val_accuracy: 0.7469 - val_loss: 0.5470
Epoch 6/50
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 552us/step - accuracy: 0.7650 - loss: 0.5192 - val_accuracy: 0.7437 - val_loss: 0.5412
Epoch 7/50
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 553us/step - accuracy: 0.7606 - loss: 0.5230 - val_accuracy: 0.7437 - val_loss: 0.5353
Epoch 8/50
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 543us/step - accuracy: 0.7635 - loss: 0.5169 - va

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [16]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
loss, accuracy = model.evaluate(X_test_scaled, y_test)

# Display the model loss and accuracy results
print(f"Model Loss: {loss:.4f}")
print(f"Model Accuracy: {accuracy:.4f}")

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7434 - loss: 0.5501 
Model Loss: 0.5672
Model Accuracy: 0.7500


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [17]:
# Set the model's file path
# Define the file path to save the model
model_file_path = Path("trained_student_loan_model.h5")

# Display the file path
print(f"Model file path set to: {model_file_path}")

# Export your model to a keras file
model.save(model_file_path)

# Display a message confirming successful save
print(f"Model saved successfully to {model_file_path}")

Model file path set to: trained_student_loan_model.h5
Model saved successfully to trained_student_loan_model.h5


---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [19]:
# Set the model's file path
model_file_path = Path("trained_student_loan_model.h5")

# Load the model to a new object
loaded_model = load_model(model_file_path)

# Verify the model structure
loaded_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 16)             │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             9 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 16)             │            32 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 516 (2.02 KB)

 Trainable params: 514 (2.01 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [20]:
# Make predictions with the test data
y_pred = (loaded_model.predict(X_test_scaled) > 0.5).astype("int32")

# Display a sample of predictions
print("Sample Predictions:", y_pred[:10].flatten())


# Display a sample of the predictions
predictions_df = pd.DataFrame({"Actual": y_test[:10], "Predicted": y_pred[:10].flatten()})
print(predictions_df)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 999us/step
Sample Predictions: [0 0 1 1 1 1 1 0 1 1]
   Actual  Predicted
0       0          0
1       1          0
2       1          1
3       1          1
4       1          1
5       1          1
6       1          1
7       0          0
8       0          1
9       0          1


In [21]:
# Save the predictions to a DataFrame and round the predictions to binary results
y_pred_probabilities = loaded_model.predict(X_test_scaled)  # Get probability outputs
y_pred = (y_pred_probabilities > 0.5).astype("int32")  # Convert to binary (0 or 1)
predictions_df = pd.DataFrame({
    "Actual": y_test,
    "Predicted": y_pred.flatten()  # Ensure correct format
})

# Display the first few rows of predictions
print(predictions_df.head())

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
   Actual  Predicted
0       0          0
1       1          0
2       1          1
3       1          1
4       1          1


### Step 4: Display a classification report with the y test data and predictions

In [22]:
# Print the classification report with the y test data and predictions
print("Classification Report:")
print(classification_report(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.75      0.74       154
           1       0.77      0.75      0.76       166

    accuracy                           0.75       320
   macro avg       0.75      0.75      0.75       320
weighted avg       0.75      0.75      0.75       320



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

## **1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**

### **Data Required to Build a Student Loan Recommendation System**
To build an **effective recommendation system** for student loan options, we need to collect **both student-specific and loan-specific data**. Below are the key categories of data that would be relevant:

---

### **1. Student Profile Data**
This data helps personalize loan recommendations based on the student's financial and academic background.
- **Student Income Level** – Determines if the student qualifies for need-based loans or grants.
- **Parental Income (if applicable)** – Helps assess financial aid eligibility.
- **Credit Score & Credit History** – Essential for private loans, as lenders evaluate creditworthiness.
- **Employment Status (if working)** – Indicates repayment ability and eligibility for income-based loans.
- **Residency Status (Domestic/International)** – Some loans are only available to domestic students.
- **Scholarships or Grants Received** – Helps determine the actual loan amount required.
- **Field of Study & Degree Type (Undergrad, Master’s, PhD)** – Some loan options are specific to certain fields or degrees.
- **University & Location** – Loan options may vary based on the institution and state regulations.

---

### **2. Loan-Specific Data**
This helps match students with the best possible loan options.
- **Loan Amount** – Determines if a federal loan is sufficient or if a private loan is needed.
- **Interest Rates (Fixed vs. Variable)** – Affects long-term repayment costs.
- **Loan Term (Years to Repay)** – Helps students select loans with manageable repayment periods.
- **Federal Loan Eligibility (Subsidized vs. Unsubsidized)** – Determines the best government loan options.
- **Repayment Options (Income-Based, Deferred, Standard, etc.)** – Matches students with flexible repayment plans.
- **Lender-Specific Requirements** – Some lenders have minimum income or cosigner requirements.

---

### **3. Behavioral & Preference Data**
This data improves recommendations based on user preferences.
- **Risk Tolerance** – Some students prefer lower monthly payments, even if total repayment costs are higher.
- **Preferred Repayment Plan** – Helps tailor loans to students who want deferment or flexible terms.
- **Previous Loan History** – Students with existing loans might need consolidation options.
- **Student Loan Awareness & Preferences** – Some students may prefer federal loans over private loans.

---

### **Why This Data is Relevant and Appropriate**
- **Personalization**: Helps match students with loans based on their financial needs and career goals.
- **Affordability**: Ensures that loan options fit the student's ability to repay.
- **Regulatory Compliance**: Certain loans are restricted based on residency, university, or degree type.
- **Better Decision-Making**: Helps students compare loans effectively based on interest rates, repayment options, and eligibility.

----

## **2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**

### **Filtering Method for the Student Loan Recommendation System**

Based on the data selected for this recommendation system, the most suitable filtering method would be **context-based filtering**.

---

### **Why Context-Based Filtering?**
Context-based filtering focuses on making recommendations based on a **user's specific attributes and situation** rather than relying on user-to-user similarities (collaborative filtering) or item similarities (content-based filtering).

The data we selected includes:
- **Student Profile Data** (income level, credit score, employment status, residency, scholarships, etc.)
- **Loan-Specific Data** (loan amount, interest rates, repayment options, eligibility criteria)
- **Behavioral & Preference Data** (risk tolerance, preferred repayment plan, previous loan history)

Since these attributes are **highly specific to an individual student**, the recommendation system will:
1. **Match loan options** based on personal financial data, academic background, and eligibility.
2. **Consider contextual factors** such as residency status, income level, and university regulations.
3. **Provide personalized suggestions** without relying on similar students' past choices (as in collaborative filtering).

---

### **Why Not Collaborative or Content-Based Filtering?**
- **Collaborative Filtering** relies on user behavior similarities, such as students with similar borrowing patterns. However, loan recommendations are **highly personal** and should be based on financial situations rather than peer borrowing behavior.
- **Content-Based Filtering** works well when comparing similar items (e.g., books, movies), but loans vary significantly in terms of eligibility, interest rates, and repayment options. This method wouldn’t be ideal for matching loans to students effectively.

---

### **Conclusion**
The **context-based filtering approach** is the best fit for this system because:
- Loan recommendations **depend on individual attributes** rather than general user behavior.
- It ensures **financial suitability and eligibility** for the student.
- It provides a **more responsible and accurate** recommendation system for student loans.



----
## **3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

### **Real-World Challenges in Building a Student Loan Recommendation System**

When developing a **student loan recommendation system**, there are several real-world challenges that must be considered to ensure fairness, accuracy, and usability. Below are two major challenges:

---

### **1. Data Privacy and Security**
#### **Why is this a concern?**
- **Sensitive Financial Data**: Student loan recommendations require **highly sensitive information**, such as income levels, credit scores, and employment status.
- **Regulatory Compliance**: The system must adhere to laws such as **FERPA (Family Educational Rights and Privacy Act)** and **GDPR (General Data Protection Regulation)** to protect student data.
- **Cybersecurity Risks**: Data breaches could expose students' financial details, leading to identity theft or fraud.

#### **How to address this challenge?**
- Implement **strong encryption** and **secure authentication** methods.
- Store minimal personal data and use **anonymous identifiers** where possible.
- Ensure compliance with **legal and ethical data protection** frameworks.

---

### **2. Fairness and Bias in Loan Recommendations**
#### **Why is this a concern?**
- **Potential Discrimination**: If the recommendation system relies heavily on factors like credit scores or income, it may **unintentionally disadvantage low-income students** or those from underrepresented backgrounds.
- **Bias in Training Data**: If historical loan approval data is biased, the model might **reinforce systemic inequalities** in lending.
- **Limited Access to Financial Education**: Some students may not understand loan terms, leading to uninformed decisions.

#### **How to address this challenge?**
- Ensure the algorithm **considers diverse financial backgrounds** and offers **fair loan options**.
- Regularly **audit and mitigate bias** in the model’s training data.
- Provide **educational resources** to help students understand loans, interest rates, and repayment options.

---

### **Conclusion**
These challenges highlight the **ethical and technical considerations** in designing a responsible student loan recommendation system. By prioritizing **data privacy** and **fairness**, we can build a system that **supports students' financial well-being** while providing accurate and secure recommendations.




